In [117]:
import os
import pathlib
import random
import shutil
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
import scipy.stats
from sklearn import metrics, preprocessing
import pandas as pd
from keras.utils import audio_dataset_from_directory
from scipy.io import wavfile
from IPython import display
import joblib

from imblearn.over_sampling import RandomOverSampler

# # Define the path to the dataset directory
# DATASET_PATH = "C:/Users/ITPKL/Desktop/pydev/ccw_sound"
# data_dir = pathlib.Path(DATASET_PATH)

# # Create training and validation dataset
# train_ds, val_ds =  audio_dataset_from_directory(
#     directory=data_dir,
#     batch_size=64,
#     validation_split=0.3,
#     seed=0,
#     output_sequence_length=16000,
#     subset='both'
# )

# # Retrieve the class names from the training dataset
# label_names = np.array(train_ds.class_names)
# label_names_val = np.array(val_ds.class_names)


# # Print the label names
# print()
# print("label names training:", label_names)
# print("label names testing:", label_names_val)

In [118]:
# Define a function to remove singleton dimensions from audio tensor
# def squeeze(audio, labels):
#     audio = tf.squeeze(audio, axis=-1)
#     return audio, labels

# # Apply the squeeze function to the training and validation datasets
# train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
# val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)

# print(train_ds)


# # Define the fixed length for FFTs
# fft_length = 1024

# # Initialize lists to store magnitude of FFTs, corresponding labels, and mask
# fft_magnitude_data = []
# labels = []
# masks = []

# # fft_magnitude_val = []
# # labels_val = []
# # masks_val = []

# Define a function to read WAV files and convert to FFT
def read_wav_file(file_path, fft_length=1024):
    try:
        sample_rate, audio = wavfile.read(file_path)
        if len(audio) == 0:
            return None
        return np.fft.fft(audio)[:fft_length]
    except Exception as e:
        print(f"Error reading WAV file: {file_path}")
        print(f"Error message: {str(e)}")
        return None


In [141]:

# Process a single WAV file
def proses_file_audio(file_path, fft_length=1024):
    fft_magnitude_data = []
    masks = []
    fft = read_wav_file(file_path, fft_length)
    if fft is not None:
        fft_magnitude = np.abs(fft)
        mask = np.ones_like(fft_magnitude)
        mask[len(fft_magnitude):] = 0
        fft_magnitude_padded = np.pad(fft_magnitude, (0, fft_length - len(fft_magnitude)))
        mask_padded = np.pad(mask, (0, fft_length - len(mask)))
        
        # Append the padded FFT magnitude and mask to their respective lists
        fft_magnitude_data.append(fft_magnitude_padded)
        masks.append(mask_padded)

    # Initialize an empty list to store the padded masks
    padded_masks = [np.pad(mask, (0, fft_length - len(mask))) for mask in masks]

    # Find the maximum length among all padded masks
    max_length = max(len(mask) for mask in padded_masks)

    # Pad each mask to the maximum length
    padded_masks = [np.pad(mask, (0, max_length - len(mask))) for mask in padded_masks]

    # Convert the lists to numpy arrays
    fft_magnitude_data = np.array(fft_magnitude_data)
    padded_masks = np.array(padded_masks)

    # Print the shapes of the resulting arrays
    print("FFT Magnitude Data Shape:", fft_magnitude_data.shape)
    print("Masks Shape:", padded_masks.shape)

    # Create a DataFrame with the magnitude of the FFT outputs and numerical labels
    fft_magnitude_df = pd.DataFrame(fft_magnitude_data)
    mask_df = pd.DataFrame(padded_masks)
    # Rename the columns of the DataFrame to fft1, fft2, ...
    # new_columns = [f'fft{n}' for n in range(1, fft_length + 1)]
    # fft_magnitude_df.columns = new_columns

    # Add a new column 'Label' to the DataFrame based on the folder name
    # fft_magnitude_df['label'] = labels

    # Display the DataFrame
    # display.display(fft_magnitude_df)

    fft_magnitude_tuples = [tuple(row) for row in fft_magnitude_df.values]
    random.shuffle(fft_magnitude_tuples)

    # Display the shuffled list of tuples
    # for tup in fft_magnitude_tuples:
    # print(tup)

    

    #sudah terbentuk numpy array  
    print(fft_magnitude_data)
    print()
    print()
    
    print(padded_masks)
    print()

    # belum terbentuk numpy array dan masih dalam bentuk dataframe
    print(fft_magnitude_df)
    print()
    print(mask_df)
    print()
    

    from sklearn.preprocessing import StandardScaler
    sc_X = StandardScaler()
    scaled_input = sc_X.fit_transform(fft_magnitude_df)

    # Load the model
    model = joblib.load('C:/Users/ITPKL/Desktop/pydev/model2024-01-30.pkl')

    # Make predictions
    y_pred = model.predict(scaled_input)
    print(y_pred)
    

In [144]:
# manggil input
proses_file_audio('ccw/ng/CCW_297.wav')

FFT Magnitude Data Shape: (1, 1024)
Masks Shape: (1, 1024)
[[5556.         3700.48378365 1121.95707109 ...  100.37376141
    79.10443937   14.2990863 ]]


[[1. 1. 1. ... 1. 1. 1.]]

     0            1            2            3            4           5     \
0  5556.0  3700.483784  1121.957071  3197.812507  1060.270372  126.851977   

        6           7           8           9     ...       1014       1015  \
0  527.65781  201.075725  178.882286  904.589214  ...  19.582697  62.648178   

        1016       1017       1018       1019       1020        1021  \
0  40.478346  18.521002  83.638313  41.777428  34.032539  100.373761   

        1022       1023  
0  79.104439  14.299086  

[1 rows x 1024 columns]

   0     1     2     3     4     5     6     7     8     9     ...  1014  \
0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  ...   1.0   

   1015  1016  1017  1018  1019  1020  1021  1022  1023  
0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  

[1 rows x 1

In [ ]:

# def dframe ():
#     # Create a DataFrame with the magnitude of the FFT outputs and numerical labels
#     fft_magnitude_df = pd.DataFrame(fft_magnitude_data)

#     # Rename the columns of the DataFrame to fft1, fft2, ...
#     # new_columns = [f'fft{n}' for n in range(1, fft_length + 1)]
#     # fft_magnitude_df.columns = new_columns

#     # Add a new column 'Label' to the DataFrame based on the folder name
#     # fft_magnitude_df['label'] = labels

#     # Display the DataFrame
#     # display.display(fft_magnitude_df)

In [ ]:
# # Takes all fft values as X and drops the Label
# X = fft_magnitude_df.drop(['label'], axis=1)
# # y = fft_magnitude_df['label']

# input_1_image = np.array(X)
# label_1_image = np.array(y)


In [ ]:

# Splits the datasset into 80% training set and 20% test set
# from sklearn.model_selection import train_test_split
# Splits the dataset into 80% training set and 20% test set
# input_latih, input_coba, label_latih, label_coba = train_test_split(input_1_image, label_1_image, test_size=0.2, random_state=0)

# Perform oversampling using RandomOverSampler
# oversampler = RandomOverSampler()
# X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)




In [ ]:
# Perform feature scaling using StandardScaler
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
input_model = sc_X.fit_transform(input_coba)


# Train the SVM (Support Vector Machine) classifier
# from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix


# load model
# classifier = SVC(kernel='poly',gamma="scale", C=2.0)
# classifier.fit(X_train_resampled, y_train_resampled)